<a href="https://colab.research.google.com/github/Behnam9473/AI/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import re
import time

Data preprocesing

Importing the dataset

In [ ]:
lines = open('/content/drive/MyDrive/NLP/movie_lines.txt', encoding = 'utf-8', errors= 'ignore').read().split('\n')
conversations = open('/content/drive/MyDrive/NLP/movie_conversations.txt', encoding = 'utf-8', errors= 'ignore').read().split('\n')


Creating a dictionary that maps each line and its id

In [ ]:
id2line = {}
for line in lines:
  _line = line.split(' +++$+++ ')
  if len(_line)==5:
    id2line[_line[0]] = _line[4]


Creating a list of all the conversitions

In [ ]:
conversations_ids = []
for conversation in conversations[:-1]:
  _converstions = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
  conversations_ids.append(_converstions.split(','))

Getting separately the questions and the answers

In [ ]:
questions = []
answers = []
for conversation in conversations_ids:
  for i in range(len(conversation) - 1):
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i+1]])

Doing a first cleaning of texts

In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r"i'm","i am", text)
  text = re.sub(r"he's","he is", text)
  text = re.sub(r"she's","she is", text)
  text = re.sub(r"that's","that is", text)
  text = re.sub(r"what's","what is", text)
  text = re.sub(r"where's","where is", text)
  text = re.sub(r"\'ll","will", text)
  text = re.sub(r"\'ve","have", text)
  text = re.sub(r"\'re","are", text)
  text = re.sub(r"\'d","whould", text)
  text = re.sub(r"won't","will not", text)
  text = re.sub(r"can't","can not", text)
  text = re.sub(r"[-()\"#@:<>{}+=~|.?,]'","", text)
  return text

#cleaning the querstions


#Cleaning the questions
clean_questtions = []
for question in questions:
  clean_questtions.append(clean_text(question))

# Clean the answers
clean_answers = []
for answer in answers:
  clean_answers.append(clean_text(answer))

Creating a dictionary that maps each word to its number of occurance

In [ ]:
word2count = {}
for question in clean_questtions:
  for word in question.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] +=1
for answer in clean_answers:
  for word in answer.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] +=1


Creating two dictionaies that map the questions words and the answers words to a unique integer

In [ ]:
thershhold = 20
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
  if count>= thershhold:
    questionswords2int[word] = word_number
    word_number +=1
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
  if count >= thershhold:
    answerswords2int[word] = word_number
    word_number +=1



Adding the last tokens to these two dictionaries

In [ ]:
# list of tokens
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']

for token in tokens:
  questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
  answerswords2int[token] = len(answerswords2int) + 1


Creating the invers dictionary of the answerwords2int dictionary

In [28]:
answers2word = {w_i: w for w, w_i in answerswords2int.items()}


Adding the End of String token to the end of every answer

In [32]:
for i in range(len(clean_answers)):
  clean_answers[i] += ' <EOS>'

Translating all questions and answers into integers
and Replaceing all the words that were filtered out by "OUT"

In [34]:
questions_to_int = []
for question in clean_questtions:
  ints = []
  for word in question.split():
    if word not in questionswords2int:
      ints.append(questionswords2int['<OUT>'])
    else:
      ints.append(questionswords2int[word])

    questions_to_int.append(ints)

answers_to_int = []
for answer in clean_answers:
  ints = []
  for word in answer.split():
    if word not in answerswords2int:
      ints.append(answerswords2int['<OUT>'])
    else:
      ints.append(answerswords2int[word])

    answers_to_int.append(ints)



Sorting question and answers by the length of questions

In [36]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1, 25+1):
  for i in enumerate(questions_to_int):
    if len(i[1]) == length:
      sorted_clean_questions.append(questions_to_int[i[0]])
      sorted_clean_answers.append(answers_to_int[i[0]])